## Generate SVG graph images recipe-wise

#### Imports

In [1]:
import io
import os
import re
import math
from xml.etree import ElementTree as ET
import urllib.parse
import json
import codecs
import subprocess
import networkx as nx
from networkx.algorithms import bipartite
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
from scipy.stats import entropy
from collections import Counter
import locale
locale.setlocale(locale.LC_ALL, 'de-DE.utf-8')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#### Read collection descriptor and ingredients catalogue

In [2]:
# namespaces
ns = {'fr': 'http://fruschtique.de/ns/recipe', 'fe': 'http://fruschtique.de/ns/fe', 'fc': 'http://fruschtique.de/ns/igt-catalog'}

# collection descriptor
desc_fn = 'C:/Users/nlutt/Documents/Websites/graphLab/currentDescriptor HD-Gemüse.xml'
desc_path = os.path.dirname(desc_fn) + '/'
with open(desc_fn, 'r', encoding='utf-8') as d:
    dd = ET.parse(d)
descriptor = dd.getroot()
#print (descriptor.find('fe:experimentPath', ns).text)

# ingredients catalogue
igtCat_fn = 'C:/Users/nlutt/myPyPro/second/data/igt_cat.json'
with open(igtCat_fn, 'r', encoding='utf-8') as i_file:
    igtCat = json.load(i_file)

#for cls in igtCat.get('classes'):
#    print (cls)

#### Make full recipes list (aka collection file)

In [3]:
# list of recipe files in collection
file_in = desc_path + descriptor.find('fe:experimentPath', ns).text + 'catalogue.xml'
with open(file_in, 'r', encoding='utf-8') as f:
    list_in = ET.parse(f)
    root_in = list_in.getroot()
in_files = [urllib.parse.unquote(doc.get("href")[8:], encoding="utf-8") for doc in root_in.findall('doc')]
#print (len(in_files))

# full recipes list
recipes = []
for fn_rcp in in_files:
    with open(fn_rcp, 'r', encoding='utf-8') as f:
        rcp_in = ET.parse(f)
        rcp_root = rcp_in.getroot()
        rcp_name = rcp_root.find('fr:recipeName', ns).text
        igdts = []
        lists = rcp_root.findall('.//fr:recipeIngredients', ns)
        for li in lists:
            xx = list(set(entry.get("ref") for entry in li.findall('.//fr:igdtName', ns) if entry.get("ref") != ''))
            igdts.extend(xx)
        rcp = {'recipeName' : rcp_name, 'ingredients' : igdts}
        recipes.append(rcp)
#print (igdts)

#### Provide full graph layout

In [4]:
# compute full graph
B = nx.Graph()
rcp_set = set()
igt_set = set()
edge_set = set()
for recipe in recipes:
    rcp_set.add(recipe.get('recipeName'))
for recipe in recipes:
    for ingredient in recipe.get('ingredients'):
        igt_set.add(ingredient)
B.add_nodes_from(rcp_set, bipartite=0)
B.add_nodes_from(igt_set, bipartite=1)
for recipe in recipes:
    rcp_name = recipe.get('recipeName')
    for ingredient in recipe.get('ingredients'):
        edge_set.add((rcp_name,ingredient))
B.add_edges_from(edge_set)
G = bipartite.weighted_projected_graph(B, igt_set)
#print (G.nodes(data=True))

# add attributes to nodes of G
  # occurence
occ_list = []
for recipe in recipes:
    rcp_igt_set = set()
    for ingredient in recipe.get('ingredients'):
        rcp_igt_set.add(ingredient)
    occ_list.extend(list(rcp_igt_set))
occ_dict = Counter(occ_list)
occ_attr = {k:{'occ':occ_dict.get(k)} for k in occ_dict.keys()}
#print (occ_attr)
nx.set_node_attributes(G, occ_attr)
  # i-name
i_name_attr = {igt:{'i-name':igtCat.get('ingredients').get(igt).get('i-name')} for igt in igt_set}
nx.set_node_attributes(G, i_name_attr)
  # i-class
i_class_attr = {igt:{'i-class':igtCat.get('ingredients').get(igt).get('i-class')} for igt in igt_set}
nx.set_node_attributes(G, i_class_attr)
#print (G.nodes(data=True))

# add attributes to edges of G
  # edge id
e_attr = {}
for e in list(G.edges(data=True)):
    x = [e[0],e[1]]
    x.sort(key=locale.strxfrm)
    id = str(x[0]) + '--' + str(x[1])
    xx = (e[0],e[1])
    e_attr[xx] = {'id':id}
nx.set_edge_attributes(G, e_attr)
#print (G.edges(data=True))
            
# create .dot file
dot = 'graph {\ngraph[rankdir="LR", outputorder="edgesfirst"]\nnode[fontname="Arial", fontsize=120, shape=circle, style=filled, fixedsize=shape];\n'
for u,v,att in G.edges(data=True):
    x = [u,v]
    x.sort(key=locale.strxfrm)
    u = x[0]
    v = x[1]
    dot += u+' -- '+v+' [penwidth='+str(att.get('weight'))
    if att.get('weight') > 1:
        dot += ', color=Red]\n'
    else:
        dot += ']\n'
for u,att in G.nodes(data=True):
    dot += u+' [width=' + str(1+3*math.sqrt(att.get('occ'))) + ', label=' + str(att.get('i-name')) + ', class=' + str(att.get('i-class')) + ']\n'
dot += '}'
dot_fn = 'C:/Users/nlutt/myPyPro/second/data/dotdot.dot'
with codecs.open(dot_fn, 'w', encoding = 'utf8') as file:
    file.write(dot)

# compute graph layout and write svg file to disc
subprocess.run (['sfdp', 'C:/Users/nlutt/myPyPro/second/data/dotdot.dot', '-o' + 'C:/Users/nlutt/myPyPro/second/data/svgGraph-poor.svg', '-Goverlap=prism', '-Tsvg'])

# read graphics file just created
with open('C:/Users/nlutt/myPyPro/second/data/svgGraph-poor.svg', 'r', encoding='utf-8') as s:
    ss = ET.parse(s)
svg_in = ss.getroot()
ns = {'svg': 'http://www.w3.org/2000/svg'}

#### Some functions

In [5]:
def gen_html_head():
    # create html head section including node styling css
    preview = ET.Element('html')
    head  = ET.SubElement(preview, 'head')
    style = ET.SubElement(head, 'style')
    style.text = \
        ' .i-alc   {fill: #7087ED; stroke: #7087ED; background-color: #7087ED}' +\
        ' .i-carb  {fill: #C8A98B; stroke: #C8A98B; background-color: #C8A98B}' +\
        ' .i-condi {fill: #D58680; stroke: #D58680; background-color: #D58680}' +\
        ' .i-egg   {fill: #70A287; stroke: #70A287; background-color: #70A287}' +\
        ' .i-etc   {fill: #9AA6BF; stroke: #9AA6BF; background-color: #9AA6BF}' +\
        ' .i-fat   {fill: #81CDD8; stroke: #81CDD8; background-color: #81CDD8}' +\
        ' .i-fish  {fill: #ffdab9; stroke: #ffdab9; background-color: #ffdab9}' +\
        ' .i-fruit {fill: #7FDD46; stroke: #7FDD46; background-color: #7FDD46}' +\
        ' .i-herb  {fill: #95A84E; stroke: #95A84E; background-color: #95A84E}' +\
        ' .i-meat  {fill: #EE5874; stroke: #EE5874; background-color: #EE5874}' +\
        ' .i-milk  {fill: #6EA2DC; stroke: #6EA2DC; background-color: #6EA2DC}' +\
        ' .i-nuts  {fill: #D09E44; stroke: #D09E44; background-color: #D09E44}' +\
        ' .i-onion {fill: #60C667; stroke: #60C667; background-color: #60C667}' +\
        ' .i-spice {fill: #FF7F50; stroke: #FF7F50; background-color: #FF7F50}' +\
        ' .i-sweet {fill: #CDE1A6; stroke: #CDE1A6; background-color: #CDE1A6}' +\
        ' .i-veg   {fill: #65DDB7; stroke: #65DDB7; background-color: #65DDB7}'
    return preview

In [6]:
def gen_html_body(preview, svg_in, ix, rcpName):
    # create html body including svg div
    body  = ET.SubElement(preview, 'body')
    div   = ET.SubElement(body,'div')
    #svg_out_attr = {'xmlns':'http://www.w3.org/2000/svg', 'xmlns:xlink':'http://www.w3.org/1999/xlink', 'version':'1.1', 'viewbox':svg_in.get('viewBox')}
    #svg_out = ET.SubElement(div, 'svg', attrib=svg_out_attr) 
    #g0_node_attr = {'id':'graph0', 'transform':svg_in.find('svg:g[@id="graph0"]',ns).get('transform')}
    #g0_node = ET.SubElement(svg_out,'g',attrib=g0_node_attr) 
    #headline = ET.SubElement(body, 'h4')
    #headline.text = f"{ix:03d} {rcpName}"
    return preview

In [7]:
def gen_svg_fontsize(scale):
    ## set svg font size
    nodes = svg_in.findall('svg:g/svg:g[@class="node"]',ns)
    xx = max([n.find('svg:ellipse',ns).get('rx') for n in nodes])
    fontsize = math.ceil(float(scale)*20.0*float(max(xx)))
    return fontsize

In [58]:
def gen_rcp_svg(G, svg_in, fontsize, rcp_name,occ_growing):
    # create svg for recipe
    vb = svg_in.get('viewBox')
    w  = vb.split()[2]
    h  = vb.split()[3]
    svg_out_attr = {'xmlns':'http://www.w3.org/2000/svg', 'xmlns:xlink':'http://www.w3.org/1999/xlink', 'version':'1.1', 'viewbox':vb, \
                    'preserveAspectRatio':'xMidYMid meet', 'zoomAndPan':'magnify', 'contentScriptType':'text/ecmascript', 'contentStyleType':'text/css', 'width':w, 'height':h}
    svg_out = ET.Element('svg', attrib=svg_out_attr)
    style = ET.SubElement(svg_out, 'style')
    style.text = \
        ' .i-alc   {fill: #7087ED; stroke: #7087ED; background-color: #7087ED}' +\
        ' .i-carb  {fill: #C8A98B; stroke: #C8A98B; background-color: #C8A98B}' +\
        ' .i-condi {fill: #D58680; stroke: #D58680; background-color: #D58680}' +\
        ' .i-egg   {fill: #70A287; stroke: #70A287; background-color: #70A287}' +\
        ' .i-etc   {fill: #9AA6BF; stroke: #9AA6BF; background-color: #9AA6BF}' +\
        ' .i-fat   {fill: #81CDD8; stroke: #81CDD8; background-color: #81CDD8}' +\
        ' .i-fish  {fill: #ffdab9; stroke: #ffdab9; background-color: #ffdab9}' +\
        ' .i-fruit {fill: #7FDD46; stroke: #7FDD46; background-color: #7FDD46}' +\
        ' .i-herb  {fill: #95A84E; stroke: #95A84E; background-color: #95A84E}' +\
        ' .i-meat  {fill: #EE5874; stroke: #EE5874; background-color: #EE5874}' +\
        ' .i-milk  {fill: #6EA2DC; stroke: #6EA2DC; background-color: #6EA2DC}' +\
        ' .i-nuts  {fill: #D09E44; stroke: #D09E44; background-color: #D09E44}' +\
        ' .i-onion {fill: #60C667; stroke: #60C667; background-color: #60C667}' +\
        ' .i-spice {fill: #FF7F50; stroke: #FF7F50; background-color: #FF7F50}' +\
        ' .i-sweet {fill: #CDE1A6; stroke: #CDE1A6; background-color: #CDE1A6}' +\
        ' .i-veg   {fill: #65DDB7; stroke: #65DDB7; background-color: #65DDB7}'
    g0_node_attr = {'id':'graph0', 'transform':svg_in.find('svg:g[@id="graph0"]',ns).get('transform')}
    g0_node = ET.SubElement(svg_out,'g',attrib=g0_node_attr)    
    rcp_g_attr = {'id':f"rr-{ix}"}
    rcp_g      = ET.SubElement(g0_node,'g',attrib=rcp_g_attr)
    name_field = ET.SubElement(rcp_g,'g')
    nf_back_attr = {'x':'0.0', 'y':str(400.0 - float(h)), 'width':str(float(w)/2), 'height':str(float(h)/24), 'fill':'white', 'stroke':'white', 'stroke-width':'1', 'fill-opacity':'1', 'stroke-opacity':'1'}
    nf_back      = ET.SubElement(name_field, 'rect', nf_back_attr) 
    nf_text_attr = {'x':'80.0', 'y':str(600.0 - float(h)), 'style':'text-anchor: start; font-family: Arial; font-size: 160px'}
    nf_text      = ET.SubElement(name_field, 'text', nf_text_attr)
    nf_text.text = f"{ix:02d} {rcp_name}"
    # recipe graph edges
    for u,v,att in G.edges(data=True):
        ed_id       = att.get('id')
        edge_attr   = {'class':'edge', 'id':ed_id, 'style':'cursor: pointer;'}
        edge        = ET.SubElement(rcp_g, 'g', attrib=edge_attr)
        title       = ET.SubElement(edge,'title')
        title.text  = ed_id
        pt          = svg_in.find (f".//svg:title[.='{ed_id}']/../svg:path", ns)
        pt_coor     = pt.get('d')
        path_attr   = {'fill':'none', 'stroke': 'black', 'd':pt_coor}
        path        = ET.SubElement(edge,'path',path_attr)
    # recipe graph nodes
    for nd in G.nodes():
        node_attr   = {'class':'node', 'id':nd, 'style':'cursor: pointer;'}
        node        = ET.SubElement(rcp_g, 'g', attrib=node_attr)  
        title       = ET.SubElement(node, 'title')
        title.text  = igtCat.get('ingredients').get(nd).get('i-name')
        
        
        #print(occ_growing)
        ell         = svg_in.find (f".//svg:title[.='{nd}']/../svg:ellipse", ns)
        x           = 36*(1 + 3*math.sqrt(occ_growing.get(nd)))
        rx          = round(x*2, 0)/2
        ry          = rx
        ell_class   = igtCat.get('ingredients').get(nd).get('i-class')
        ell_attr    = {'class':f"i-{ell_class}", 'cx':ell.get('cx'), 'cy':ell.get('cy'), 'rx':str(rx), 'ry':str(ry)}
        ellipse     = ET.SubElement(node, 'ellipse', attrib=ell_attr)
        txt         = svg_in.find (f".//svg:title[.='{nd}']/../svg:text", ns)
        text_attr   = {'x':txt.get('x'), 'y':txt.get('y'), 'style':f"text-anchor: middle; font-family: Arial; font-size: {fontsize}px;"}
        text        = ET.SubElement(node, 'text', attrib=text_attr)
        text.text   = txt.text    
    return svg_out

#### Create recipe graphs and their svg representations

In [59]:
###
# generate svg per recipe and collect svg files in directory
### 

K  = nx.Graph()                             # recipe graph (complete graph)
ix = 1
rcp_name = ''
fontsize = gen_svg_fontsize (1.0)
occ_growing = {k:0 for k in occ_dict.keys() }
#print (occ_growing)

# compute occurence values
occ_list = []
for recipe in recipes:
    rcp_igt_set = set()
    for ingredient in recipe.get('ingredients'):
        rcp_igt_set.add(ingredient)
    occ_list.extend(list(rcp_igt_set))
occ_dict = Counter(occ_list)
#print (occ_dict)
occ_attr = {k:{'occ':occ_dict.get(k)} for k in occ_dict.keys()}
    
# loop over recipes in collection for creating recipe graphs and adding them to ingredient graph
for recipe in recipes:                      # collect ingredients for recipe graph
    rcp_name = recipe.get('recipeName')
    igt_set = set()                         # use set type for ingredients -> no duplicate entries
    for ingredient in recipe.get('ingredients'):
        igt_set.add(ingredient)
    K = nx.complete_graph(igt_set)          # build recipe graph
    # add attributes to nodes of K
        # occurence
    nx.set_node_attributes(K, occ_attr)
    for k in occ_growing.keys():
        if k in igt_set:
            occ_growing[k] += 1
    #print (occ_growing)
        # i-name
    i_name_attr = {igt:{'i-name':igtCat.get('ingredients').get(igt).get('i-name')} for igt in igt_set}
    nx.set_node_attributes(K, i_name_attr)
        # i-class
    i_class_attr = {igt:{'i-class':igtCat.get('ingredients').get(igt).get('i-class')} for igt in igt_set}
    nx.set_node_attributes(K, i_class_attr)
    # add attributes to edges of G
        # edge id
    e_attr = {}
    for e in list(K.edges(data=True)):
        x = [e[0],e[1]]
        x.sort(key=locale.strxfrm)
        id = str(x[0]) + '--' + str(x[1])
        xx = (e[0],e[1])
        e_attr[xx] = {'id':id}
    nx.set_edge_attributes(K, e_attr)
    
    #print (K.nodes(data=True))
    build = gen_rcp_svg (K,svg_in,fontsize,rcp_name,occ_growing)
    ix += 1
    tree = ET.ElementTree(build)
    ET.indent(tree)
    tree.write(f"C:/Users/nlutt/Desktop/rcp_graphs/{rcp_name}.svg")

#### Grow html-enclosed svg graph recipe-wise

In [10]:
###
# generate full graph
###

#IG = nx.Graph()                             # ingredient graph
K  = nx.Graph()                             # recipe graph (complete graph)
ix = 1
rcp_name = ''

build    = gen_html_head()
build    = gen_html_body (build, ix, '')
build    = gen_svg_div (build, svg_in)
fontsize = gen_svg_fontsize (1.0)
#ET.dump (build)

# loop over recipes in collection for creating recipe graphs and adding them to ingredient graph
for recipe in recipes:                      # collect ingredients for recipe graph
    rcp_name = recipe.get('recipeName')
    igt_set = set()                         # use set type for ingredients -> no duplicates
    for ingredient in recipe.get('ingredients'):
        igt_set.add(ingredient)
    K = nx.complete_graph(igt_set)          # build recipe graph
    # add attributes to nodes of K
        # occurence
    occ_list = []
    for recipe in recipes:
        rcp_igt_set = set()
        for ingredient in recipe.get('ingredients'):
            rcp_igt_set.add(ingredient)
        occ_list.extend(list(rcp_igt_set))
    occ_dict = Counter(occ_list)
    occ_attr = {k:{'occ':occ_dict.get(k)} for k in occ_dict.keys()}
    nx.set_node_attributes(K, occ_attr)
        # i-name
    i_name_attr = {igt:{'i-name':igtCat.get('ingredients').get(igt).get('i-name')} for igt in igt_set}
    nx.set_node_attributes(K, i_name_attr)
        # i-class
    i_class_attr = {igt:{'i-class':igtCat.get('ingredients').get(igt).get('i-class')} for igt in igt_set}
    nx.set_node_attributes(K, i_class_attr)
    # add attributes to edges of G
        # edge id
    e_attr = {}
    for e in list(K.edges(data=True)):
        x = [e[0],e[1]]
        x.sort(key=locale.strxfrm)
        id = str(x[0]) + '--' + str(x[1])
        xx = (e[0],e[1])
        e_attr[xx] = {'id':id}
    nx.set_edge_attributes(K, e_attr)
    build = gen_rcp_svg (build,K,svg_in,fontsize,rcp_name)
    ix += 1
tree = ET.ElementTree(build)
ET.indent(tree)
tree.write('C:/Users/nlutt/Desktop/build.html')
  
    # weighted union of recipe graph and ingredient graph
    #for u,v,attrib in K.edges(data=True):   # loop over edges of recipe graph
        #if IG.has_edge(u,v):
            #IG[u][v]['weight'] += 1         # increase weight of ingredient graph edge
        #else:
            #IG.add_edge(u, v, weight=1)     # add new edge to ingredient graph
    #gen_svg(IG,recipe,ix)

In [9]:
def xx_gen_rcp_svg(preview, G, svg_in, fontsize,rcp_name):
    # enclosing element for recipe graph
    g0 = preview.find(".//g[@id='graph0']",ns)
    
    rcp_g_attr = {'id':f"rr-{ix}", 'transform':'transform_rcp_graph', 'visibility':'hidden'}
    rcp_g      = ET.SubElement(g0,'g',attrib=rcp_g_attr)
    
    if ix == 1:
        begin = '0.1s'
    else:
        begin = f"anim{ix-1:03d}.end+3s"
    anim_attr  = {'id':f"anim{ix:03d}", 'attributeName':'visibility', 'attributeType':'XML', 'from':'hidden', 'to':'visible', 'begin':begin, 'dur':'0.1s', 'fill':'freeze'}
    anim       = ET.SubElement(rcp_g,'animate',attrib=anim_attr)
    rcp_name_attr = {"font-family": "Arial", "font-size": f"{fontsize}px", "visibility":"hidden"}
    rcp_name_node   = ET.SubElement(rcp_g,'text', attrib=rcp_name_attr)
    rcp_name_node.text = rcp_name
    if ix == 1:
        begin_text = '0.1s'
    else:
        begin_text = f"anim{ix-1:03d}.end"
    #begin_text = f"anim{ix-1:03d}.end"
    anim_text_attr  = {'id':f"txt{ix:03d}", 'attributeName':'visibility', 'attributeType':'XML', 'from':'hidden', 'to':'visible', 'begin':begin_text, 'dur':'3s'}
    anim_text       = ET.SubElement(rcp_name_node,'animate',attrib=anim_text_attr)
    
    
    # recipe graph edges
    for u,v,att in G.edges(data=True):
        ed_id       = att.get('id')
        edge_attr   = {'class':'edge', 'id':ed_id, 'style':'cursor: pointer;'}
        edge        = ET.SubElement(rcp_g, 'g', attrib=edge_attr)
        title       = ET.SubElement(edge,'title')
        title.text  = ed_id
        pt          = svg_in.find (f".//svg:title[.='{ed_id}']/../svg:path", ns)
        pt_coor     = pt.get('d')
        path_attr   = {'fill':'none', 'stroke': 'black', 'd':pt_coor}
        path        = ET.SubElement(edge,'path',path_attr)
    # recipe graph nodes
    for nd in G.nodes():
        node_attr   = {'class':'node', 'id':nd, 'style':'cursor: pointer;'}
        node        = ET.SubElement(rcp_g, 'g', attrib=node_attr)  
        title       = ET.SubElement(node, 'title')
        title.text  = igtCat.get('ingredients').get(nd).get('i-name')
        ell         = svg_in.find (f".//svg:title[.='{nd}']/../svg:ellipse", ns)
        ell_class   = igtCat.get('ingredients').get(nd).get('i-class')
        ell_attr    = {'class':f"i-{ell_class}", 'cx':ell.get('cx'), 'cy':ell.get('cy'), 'rx':ell.get('rx'), 'ry':ell.get('ry')}
        ellipse     = ET.SubElement(node, 'ellipse', attrib=ell_attr)
        txt         = svg_in.find (f".//svg:title[.='{nd}']/../svg:text", ns)
        text_attr   = {'x':txt.get('x'), 'y':txt.get('y'), 'style':f"text-anchor: middle; font-family: Arial; font-size: {fontsize}px;"}
        text        = ET.SubElement(node, 'text', attrib=text_attr)
        text.text   = txt.text    
    return preview
    

#### Playground

In [16]:
<svg viewBox="0 0 220 120" xmlns="http://www.w3.org/2000/svg">
	
	<g stroke="seagreen" stroke-width="5" fill="skyblue" opacity="0">
		<animate id="g1" attributeName="opacity" attributeType="XML" begin="2s" dur="1s" 
			fill="freeze" from="0" to="1"/> 
		<rect
			x="10"
			y="10"
			width="200"
			height="100"
			stroke="black"
			stroke-width="5"
			fill="transparent" />
		<rect x="20" y="20" width="40" height="80" opacity="0" >
			<animate id="a1" attributeName="opacity" attributeType="XML" begin="g1.end+3s" dur="1s" 
				fill="freeze" from="0" to="1"/> 
		</rect>
		<rect x="65" y="20" width="40" height="80" opacity="0" >
			<animate id="a2" attributeName="opacity" attributeType="XML" begin="a1.end+3s" dur="1s" 
				fill="freeze" from="0" to="1"/> 
		</rect>
		<rect x="110" y="20" width="40" height="80" opacity="0" >
			<animate id="a3" attributeName="opacity" attributeType="XML" begin="a2.end+3s" dur="1s" 
				fill="freeze" from="0" to="1"/> 
		</rect>
	</g>
</svg>

Graph with 100 nodes and 778 edges
lauch
